# **Step 1: Import required libraries**

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import seaborn as sns # for plot visualization
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error

# **Step 1 : Load the Data**

In [ ]:
weather_df = pd.read_csv('/kaggle/input/bangladesh-historical-weather-dataset-2008-2023/Bangladesh Historical Weather Dataset 2008-2023.csv', parse_dates=['time'], index_col='time')
weather_df.head()

# **Step 2: Feature Engineering**

In [ ]:
weather_df = weather_df.loc[:,['precipitation_sum (mm)', 'sunrise (iso8601)',
                               'sunset (iso8601)', 'temperature_2m_max (°C)', 
                               'temperature_2m_min (°C)', 'rain_sum (mm)', 
                               'temperature_2m_mean (°C)', 'snowfall_sum (cm)']]

weather_df = weather_df.rename(index=str, columns={'precipitation_sum (mm)': 'precipitation',
                                                   'sunrise (iso8601)': 'sunrise', 
                                                   'sunset (iso8601)': 'sunset', 
                                                   'temperature_2m_max (°C)':'temp_max',
                                                   'temperature_2m_min (°C)':'temp_min',
                                                   'temperature_2m_mean (°C)': 'temprature',
                                                   'rain_sum (mm)':'rain',
                                                  'snowfall_sum (cm)':'snowfall'})

print(f'dataset shape (rows, columns) - {weather_df.shape}')
weather_df.head()

In [ ]:
# weather_df['temprature'] = (weather_df['temp_max'] + weather_df['temp_min']) / 2

# weather_df.head()

In [ ]:
# lets check dtype of all columns, 
weather_df.dtypes, weather_df.index.dtype

It shows 'index' as object type which needs to be converted to datetime otherwise we won't be able to perform scaling during time series analysis.

In [ ]:
weather_df.index = pd.to_datetime(weather_df.index)
weather_df.index

# **Step 3 : Data Preprocessing**

In [ ]:
def list_and_visualize_missing_data(dataset):
    # Listing total null items and its percent with respect to all nulls
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = ((dataset.isnull().sum())/(dataset.isnull().count())).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    missing_data = missing_data[missing_data.Total >= 0]
    #missing_data = missing_data[missing_data.Total > 0]

    missing_data.plot.bar(subplots=True, figsize=(16,9))

list_and_visualize_missing_data(weather_df)

In [ ]:
# will fill with previous valid value
#weather_df.ffill(inplace=True)
weather_df=weather_df.dropna(axis=0)
weather_df[weather_df.isnull()].count()

In [ ]:
weather_df.describe()

In [ ]:
# weather_df = weather_df[weather_df.temprature < 50]
# #weather_df = weather_df[weather_df.humidity <= 100]

# **Step 4 : Exploratory Data Analysis & Visualizations**

In [ ]:
weather_condition = (weather_df.temprature.value_counts()/(weather_df.temprature.value_counts().sum()))*100
weather_condition.plot.bar(figsize=(16,9))
plt.xlabel('Temperature')
plt.ylabel('Percent')

In [ ]:
weather_df.plot(subplots=True, figsize=(20,12))

In [ ]:
weather_df['2008':'2024'].resample('D').fillna(method='pad').plot(subplots=True, figsize=(20,12))

In [ ]:
weather_df.drop(columns=['sunrise', 'sunset'], inplace=True)
weather_df.head()

# **Step 5 : Split the dataset into training and testing sets**

In [ ]:
train_df = weather_df['2008':'2020'].resample('M').mean().fillna(method='pad')

train_df.drop(columns='precipitation', axis=1, inplace=True)
train_df.drop(columns='temp_max', axis=1, inplace=True)
train_df.drop(columns='temp_min', axis=1, inplace=True)
train_df.drop(columns='rain', axis=1, inplace=True)
train_df.drop(columns='snowfall', axis=1, inplace=True)


test_df = weather_df['2020':'2024'].resample('M').mean().fillna(method='pad')

test_df.drop(columns='precipitation', axis=1, inplace=True)
test_df.drop(columns='temp_max', axis=1, inplace=True)
test_df.drop(columns='temp_min', axis=1, inplace=True)
test_df.drop(columns='rain', axis=1, inplace=True)
test_df.drop(columns='snowfall', axis=1, inplace=True)


In [ ]:
train_df.head()

In [ ]:
# check rolling mean and rolling standard deviation
def plot_rolling_mean_std(ts):
    rolling_mean = ts.rolling(12).mean()
    rolling_std = ts.rolling(12).std()
    plt.figure(figsize=(22,10))

    plt.plot(ts, label='Actual Mean')
    plt.plot(rolling_mean, label='Rolling Mean')
    plt.plot(rolling_std, label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Rolling Standard Deviation')
    plt.legend()
    plt.show()

In [ ]:
# Augmented Dickey–Fuller test
def perform_dickey_fuller_test(ts):
    result = adfuller(ts, autolag='AIC')
    print('Test statistic: ' , result[0])
    print('Critical Values:' ,result[4])

In [ ]:
# check stationary: mean, variance(std)and adfuller test
plot_rolling_mean_std(train_df.temprature)
perform_dickey_fuller_test(train_df.temprature)

In [ ]:
# Original Series
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(train_df.values); 
axes[0, 0].set_title('Original Series')
plot_acf(train_df.values, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(train_df.temprature.diff().values); 
axes[1, 0].set_title('1st Order Differencing')
plot_acf(train_df.diff().dropna().values,ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(train_df.temprature.diff().diff().values); 
axes[2, 0].set_title('2nd Order Differencing')
plot_acf(train_df.diff().diff().dropna().values,ax=axes[2, 1])

plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(train_df.diff().values); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,5))
plot_pacf(train_df.diff().dropna().values, ax=axes[1])

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(train_df.diff().values); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,1.2))
plot_acf(train_df.diff().dropna().values, ax=axes[1])

plt.show()

In [ ]:
acf_lag = acf(train_df.diff().dropna().values, nlags=20)
pacf_lag = pacf(train_df.diff().dropna().values, nlags=20, method='ols')

plt.figure(figsize=(22,10))

plt.subplot(121)
plt.plot(acf_lag)
plt.axhline(y=0,linestyle='--',color='silver')
plt.axhline(y=-1.96/np.sqrt(len(train_df.diff().values)),linestyle='--',color='silver')
plt.axhline(y=1.96/np.sqrt(len(train_df.diff().values)),linestyle='--',color='silver')
plt.title("Autocorrelation Function")

plt.subplot(122)
plt.plot(pacf_lag)
plt.axhline(y=0,linestyle='--',color='silver')
plt.axhline(y=-1.96/np.sqrt(len(train_df.diff().values)),linestyle='--',color='silver')
plt.axhline(y=1.96/np.sqrt(len(train_df.diff().values)),linestyle='--',color='silver')
plt.title("Partial Autocorrelation Function")
plt.tight_layout()

# **Step 6 : Model Building and Training**

In [ ]:
#default order=(2,0,2), thesis paper order=(2,1,1)
model = ARIMA(train_df.values, order=(2,0,2))
model_fit = model.fit(disp=0)
print(model_fit.summary())

# **Step 7 : Predictions and Model evaluation**

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
# Actual vs Fitted
model_fit.plot_predict(dynamic=False)
plt.show()

# **Step 8 : Weather Forecast**

In [ ]:
# # Forecast
fc, se, conf = model_fit.forecast(48, alpha=0.05)

print(fc)
# Make as pandas series
fc_series = pd.Series(fc, index=test_df.index)
lower_series = pd.Series(conf[:, 0], index=test_df.index)
upper_series = pd.Series(conf[:, 1], index=test_df.index)

# # Plot
plt.figure(figsize=(16,5), dpi=100)
plt.plot(train_df, label='training')
plt.plot(test_df, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()
# test_df.index

In [ ]:
# plot Testing and Forecasted data
plt.plot(test_df, label='Actual')
plt.plot(fc_series, label='Forecast', color='red')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# Display the forecast values with dates
forecast_result = pd.DataFrame({'Date': test_df.index, 'Actual': test_df.values.flatten(), 'Forecast': fc_series.values})
print(forecast_result)

# **Step 9 : Calculate the evaluation metrices**

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Assuming you have your true values and predicted values
true_values = test_df  # Replace with your true values
predicted_values = fc_series  # Replace with your predicted values

# Calculate MSE
mse = mean_squared_error(test_df, fc_series)
print('Test Mean Squared Error (MSE): ',mse)

# Calculate MAE
mae = mean_absolute_error(true_values, predicted_values)
print('Mean Absolute Error (MAE): ', mae)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
print('Root Mean Squared Error (RMSE): ', rmse)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(true_values, predicted_values)
print('Mean Absolute Percentage Error (MAPE): ', mape)


In [ ]:
test_df.head()

In [ ]:
fc_series.head()

In [ ]:
# Assuming 'test_df' and 'fc_series' have the same index
result_df = pd.DataFrame({
    'Date': test_df.index,
    'Actual': test_df.values.flatten(),  # Flatten to 1D array
    'Predicted': fc_series.values.flatten(),  # Flatten to 1D array
    'Lower Bound': lower_series.values.flatten(),  # Flatten to 1D array
    'Upper Bound': upper_series.values.flatten()  # Flatten to 1D array
})

# Display the DataFrame
print(result_df)

# **Rainfall**

In [ ]:
weather_df.head()

In [ ]:
# will fill with previous valid value
weather_df.ffill(inplace=True)
weather_df[weather_df.isnull()].count()

In [ ]:
train_df = weather_df['2008':'2020'].resample('M').mean().fillna(method='pad')

train_df.drop(columns='precipitation', axis=1, inplace=True)
train_df.drop(columns='temp_max', axis=1, inplace=True)
train_df.drop(columns='temp_min', axis=1, inplace=True)
train_df.drop(columns='temprature', axis=1, inplace=True)
train_df.drop(columns='snowfall', axis=1, inplace=True)

test_df = weather_df['2020':'2024'].resample('M').mean().fillna(method='pad')

test_df.drop(columns='precipitation', axis=1, inplace=True)
test_df.drop(columns='temp_max', axis=1, inplace=True)
test_df.drop(columns='temp_min', axis=1, inplace=True)
test_df.drop(columns='temprature', axis=1, inplace=True)
test_df.drop(columns='snowfall', axis=1, inplace=True)

In [ ]:
train_df.head()

In [ ]:
# check rolling mean and rolling standard deviation
def plot_rolling_mean_std(ts):
    rolling_mean = ts.rolling(12).mean()
    rolling_std = ts.rolling(12).std()
    plt.figure(figsize=(22,10))

    plt.plot(ts, label='Actual Mean')
    plt.plot(rolling_mean, label='Rolling Mean')
    plt.plot(rolling_std, label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Rolling Standard Deviation')
    plt.legend()
    plt.show()

In [ ]:
# Augmented Dickey–Fuller test
def perform_dickey_fuller_test(ts):
    result = adfuller(ts, autolag='AIC')
    print('Test statistic: ' , result[0])
    print('Critical Values:' ,result[4])

In [ ]:
# check stationary: mean, variance(std)and adfuller test
plot_rolling_mean_std(train_df.rain)
perform_dickey_fuller_test(train_df.rain)

In [ ]:
# Original Series
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(train_df.values); 
axes[0, 0].set_title('Original Series')
plot_acf(train_df.values, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(train_df.rain.diff().values); 
axes[1, 0].set_title('1st Order Differencing')
plot_acf(train_df.diff().dropna().values,ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(train_df.rain.diff().diff().values); 
axes[2, 0].set_title('2nd Order Differencing')
plot_acf(train_df.diff().diff().dropna().values,ax=axes[2, 1])

plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(train_df.diff().values); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,5))
plot_pacf(train_df.diff().dropna().values, ax=axes[1])

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(train_df.diff().values); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,1.2))
plot_acf(train_df.diff().dropna().values, ax=axes[1])

plt.show()

In [ ]:
acf_lag = acf(train_df.diff().dropna().values, nlags=20)
pacf_lag = pacf(train_df.diff().dropna().values, nlags=20, method='ols')

plt.figure(figsize=(22,10))

plt.subplot(121)
plt.plot(acf_lag)
plt.axhline(y=0,linestyle='--',color='silver')
plt.axhline(y=-1.96/np.sqrt(len(train_df.diff().values)),linestyle='--',color='silver')
plt.axhline(y=1.96/np.sqrt(len(train_df.diff().values)),linestyle='--',color='silver')
plt.title("Autocorrelation Function")

plt.subplot(122)
plt.plot(pacf_lag)
plt.axhline(y=0,linestyle='--',color='silver')
plt.axhline(y=-1.96/np.sqrt(len(train_df.diff().values)),linestyle='--',color='silver')
plt.axhline(y=1.96/np.sqrt(len(train_df.diff().values)),linestyle='--',color='silver')
plt.title("Partial Autocorrelation Function")
plt.tight_layout()

In [ ]:
#default order=(2,0,2), thesis paper order=(2,1,1)
model = ARIMA(train_df.values, order=(2,0,2))
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
# Actual vs Fitted
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
# # Forecast
fc, se, conf = model_fit.forecast(24, alpha=0.05)

# print(fc)
# Make as pandas series
fc_series = pd.Series(fc, index=test_df.index)
lower_series = pd.Series(conf[:, 0], index=test_df.index)
upper_series = pd.Series(conf[:, 1], index=test_df.index)

# # Plot
plt.figure(figsize=(16,5), dpi=100)
plt.plot(train_df, label='training')
plt.plot(test_df, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()
# test_df.index

In [ ]:
# plot Testing and Forecasted data
plt.plot(test_df, label='Actual')
plt.plot(fc_series, label='Forecast', color='red')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# Assuming you have your true values and predicted values
true_values = test_df  # Replace with your true values
predicted_values = fc_series  # Replace with your predicted values

# Calculate MSE
mse = mean_squared_error(test_df, fc_series)
print('Test Mean Squared Error (MSE): ',mse)

# Calculate MAE
mae = mean_absolute_error(true_values, predicted_values)
print('Mean Absolute Error (MAE): ', mae)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
print('Root Mean Squared Error (RMSE): ', rmse)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(true_values, predicted_values)
print('Mean Absolute Percentage Error (MAPE): ', mape)

In [ ]:
# Assuming 'test_df' and 'fc_series' have the same index
result_df = pd.DataFrame({
    'Date': test_df.index,
    'Actual': test_df.values.flatten(),  # Flatten to 1D array
    'Predicted': fc_series.values.flatten(),  # Flatten to 1D array
    'Lower Bound': lower_series.values.flatten(),  # Flatten to 1D array
    'Upper Bound': upper_series.values.flatten()  # Flatten to 1D array
})

# Display the DataFrame
print(result_df)